# 🧍 Human Body Reconstruction Pipeline v2.1 (Fixed)

## Production-Grade Implementation - Colab Compatible

### Fixed Issues:
- ✅ No MMPose (causes Python 3.12 conflicts)
- ✅ Uses MediaPipe for pose (stable, fast)
- ✅ Clean dependency installation
- ✅ Iterative PnP camera estimation
- ✅ PCA-based circumference measurements

---
# STEP 0: Setup (Run All Cells in Order)
---

In [ ]:
#@title 0.1 Check GPU
!nvidia-smi

import torch
print(f"\n🔥 PyTorch: {torch.__version__}")
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
#@title 0.2 Install Dependencies (Fixed - No MMPose)
#@markdown This avoids the Python 3.12 compatibility issues

# Core packages (these are safe)
!pip install -q opencv-python-headless
!pip install -q mediapipe==0.10.9
!pip install -q ultralytics
!pip install -q smplx
!pip install -q chumpy
!pip install -q trimesh
!pip install -q scikit-learn

print("\n✅ Dependencies installed successfully!")

In [ ]:
#@title 0.3 Verify Imports
import torch
import numpy as np
import cv2
import mediapipe as mp
from ultralytics import YOLO
import smplx
import trimesh
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

print("✅ All imports successful!")
print(f"   PyTorch: {torch.__version__}")
print(f"   OpenCV: {cv2.__version__}")
print(f"   MediaPipe: {mp.__version__}")

In [ ]:
#@title 0.4 Create Directories
import os

for d in ['workspace/input', 'workspace/output/frames', 'workspace/output/final', 'workspace/models/smplx']:
    os.makedirs(d, exist_ok=True)

print("✅ Directories created")

---
# STEP 1: Upload Video
---

In [ ]:
#@title 1.1 Upload Your Video
#@markdown Click "Choose Files" button and select your turntable video

from google.colab import files

print("📤 Upload your turntable video:")
print("   Requirements:")
print("   - 5-10 seconds long")
print("   - Person rotating slowly (or camera moving around person)")
print("   - Full body visible")
print("   - MP4 format recommended")
print("")

uploaded = files.upload()

VIDEO_PATH = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {VIDEO_PATH}")

In [ ]:
#@title 1.2 Verify Video
import cv2
import matplotlib.pyplot as plt

cap = cv2.VideoCapture(VIDEO_PATH)

# Get video info
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = total_frames / fps if fps > 0 else 0

print(f"📹 Video Info:")
print(f"   Resolution: {width} x {height}")
print(f"   Duration: {duration:.1f} seconds")
print(f"   FPS: {fps:.0f}")
print(f"   Total Frames: {total_frames}")

# Show first frame
ret, frame = cap.read()
if ret:
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('First Frame Preview')
    plt.axis('off')
    plt.show()
    print("\n✅ Video loaded successfully!")
else:
    print("\n❌ Error: Could not read video")

cap.release()

# Store for later
VIDEO_INFO = {'width': width, 'height': height, 'fps': fps, 'total_frames': total_frames}

---
# STEP 2: Extract Frames
---

In [ ]:
#@title 2.1 Extract 8 Strategic Frames
import cv2
import numpy as np

N_FRAMES = 8  #@param {type:"slider", min:4, max:16, step:2}

cap = cv2.VideoCapture(VIDEO_PATH)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Get evenly spaced frame indices
indices = np.linspace(0, total_frames - 1, N_FRAMES, dtype=int)

frames = []
print(f"📸 Extracting {N_FRAMES} frames...")

for i, idx in enumerate(indices):
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ret, frame = cap.read()
    
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append({
            'index': i,
            'frame_idx': int(idx),
            'data': frame_rgb
        })
        print(f"   Frame {i+1}/{N_FRAMES} extracted (original index: {idx})")

cap.release()
print(f"\n✅ Extracted {len(frames)} frames")

In [ ]:
#@title 2.2 Preview Extracted Frames
import matplotlib.pyplot as plt

cols = 4
rows = (len(frames) + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
axes = axes.flatten() if len(frames) > cols else [axes] if len(frames) == 1 else axes

for i, ax in enumerate(axes):
    if i < len(frames):
        ax.imshow(frames[i]['data'])
        ax.set_title(f"Frame {i+1}")
    ax.axis('off')

plt.tight_layout()
plt.show()

---
# STEP 3: Person Detection & Pose Estimation
---

In [ ]:
#@title 3.1 Initialize Detection & Pose Models
from ultralytics import YOLO
import mediapipe as mp

# YOLOv8 for bounding box detection
print("🔄 Loading YOLOv8...")
detector = YOLO('yolov8x.pt')
print("✅ YOLOv8 loaded")

# MediaPipe for pose estimation
print("\n🔄 Loading MediaPipe Pose...")
mp_pose = mp.solutions.pose
pose_estimator = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,  # Highest accuracy
    enable_segmentation=True,
    min_detection_confidence=0.5
)
print("✅ MediaPipe Pose loaded")

In [ ]:
#@title 3.2 Run Detection & Pose on All Frames
import numpy as np

# MediaPipe to COCO keypoint mapping
MP_TO_COCO = {
    0: 0,    # nose
    2: 1,    # left_eye
    5: 2,    # right_eye
    7: 3,    # left_ear
    8: 4,    # right_ear
    11: 5,   # left_shoulder
    12: 6,   # right_shoulder
    13: 7,   # left_elbow
    14: 8,   # right_elbow
    15: 9,   # left_wrist
    16: 10,  # right_wrist
    23: 11,  # left_hip
    24: 12,  # right_hip
    25: 13,  # left_knee
    26: 14,  # right_knee
    27: 15,  # left_ankle
    28: 16,  # right_ankle
}

def process_frame(image, detector, pose_estimator):
    """Detect person and estimate pose"""
    h, w = image.shape[:2]
    
    # Step 1: Detect bounding box
    det_results = detector(image, verbose=False, classes=[0])  # class 0 = person
    
    if len(det_results[0].boxes) == 0:
        return None
    
    # Get largest person bbox
    boxes = det_results[0].boxes
    areas = (boxes.xyxy[:, 2] - boxes.xyxy[:, 0]) * (boxes.xyxy[:, 3] - boxes.xyxy[:, 1])
    best_idx = areas.argmax()
    bbox = boxes.xyxy[best_idx].cpu().numpy()
    
    # Step 2: Crop and run pose estimation
    x1, y1, x2, y2 = map(int, bbox)
    pad = int(max(x2-x1, y2-y1) * 0.1)
    x1, y1 = max(0, x1-pad), max(0, y1-pad)
    x2, y2 = min(w, x2+pad), min(h, y2+pad)
    
    crop = image[y1:y2, x1:x2]
    results = pose_estimator.process(crop)
    
    if not results.pose_landmarks:
        return None
    
    # Extract keypoints
    ch, cw = crop.shape[:2]
    mp_keypoints = []
    for lm in results.pose_landmarks.landmark:
        mp_keypoints.append([lm.x * cw + x1, lm.y * ch + y1, lm.visibility])
    mp_keypoints = np.array(mp_keypoints)
    
    # Convert to COCO format
    coco_keypoints = np.zeros((17, 3))
    for mp_idx, coco_idx in MP_TO_COCO.items():
        coco_keypoints[coco_idx] = mp_keypoints[mp_idx]
    
    # Get segmentation mask
    segmentation = None
    if results.segmentation_mask is not None:
        seg_crop = (results.segmentation_mask > 0.5).astype(np.uint8)
        segmentation = np.zeros((h, w), dtype=np.uint8)
        segmentation[y1:y2, x1:x2] = seg_crop
    
    return {
        'bbox': bbox,
        'keypoints': coco_keypoints,
        'segmentation': segmentation
    }

# Process all frames
print(f"🔍 Processing {len(frames)} frames...")
pose_results = []

for i, frame in enumerate(frames):
    result = process_frame(frame['data'], detector, pose_estimator)
    
    if result:
        result['frame_index'] = i
        pose_results.append(result)
        print(f"   Frame {i+1}: ✓ Person detected, 17 keypoints extracted")
    else:
        pose_results.append(None)
        print(f"   Frame {i+1}: ⚠️ No detection")

success_count = sum(1 for r in pose_results if r is not None)
print(f"\n✅ Processing complete: {success_count}/{len(frames)} successful")

In [ ]:
#@title 3.3 Visualize Pose Estimation
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# COCO skeleton connections
SKELETON = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # Head
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Arms
    (5, 11), (6, 12), (11, 12),  # Torso
    (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
]

cols = 4
rows = (len(frames) + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < len(frames):
        ax.imshow(frames[i]['data'])
        
        if pose_results[i] is not None:
            kp = pose_results[i]['keypoints']
            bbox = pose_results[i]['bbox']
            
            # Draw bbox
            rect = patches.Rectangle(
                (bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1],
                linewidth=2, edgecolor='lime', facecolor='none'
            )
            ax.add_patch(rect)
            
            # Draw keypoints
            for j, (x, y, c) in enumerate(kp):
                if c > 0.3:
                    ax.scatter(x, y, c='red', s=30, zorder=5)
            
            # Draw skeleton
            for (s, e) in SKELETON:
                if kp[s, 2] > 0.3 and kp[e, 2] > 0.3:
                    ax.plot([kp[s, 0], kp[e, 0]], [kp[s, 1], kp[e, 1]], 'c-', lw=2)
            
            ax.set_title(f"Frame {i+1}: ✓")
        else:
            ax.set_title(f"Frame {i+1}: ⚠️")
    ax.axis('off')

plt.tight_layout()
plt.show()

---
# STEP 4: Upload SMPL-X Model
---

In [ ]:
#@title 4.1 Upload SMPL-X Model
#@markdown Download from https://smpl-x.is.tue.mpg.de/ (free registration)
#@markdown Upload the SMPLX_NEUTRAL.npz file

import os
from google.colab import files

SMPLX_PATH = 'workspace/models/smplx'
model_file = os.path.join(SMPLX_PATH, 'SMPLX_NEUTRAL.npz')

if not os.path.exists(model_file):
    print("📤 Upload SMPLX_NEUTRAL.npz:")
    print("   (Download from https://smpl-x.is.tue.mpg.de/)")
    print("")
    
    uploaded = files.upload()
    
    for fname in uploaded.keys():
        dest = os.path.join(SMPLX_PATH, fname)
        os.rename(fname, dest)
        print(f"\n✅ Saved to: {dest}")
else:
    print(f"✅ SMPL-X model already exists at {model_file}")

In [ ]:
#@title 4.2 Load SMPL-X Model
import smplx
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔄 Loading SMPL-X on {device}...")

body_model = smplx.create(
    SMPLX_PATH,
    model_type='smplx',
    gender='neutral',
    use_face_contour=True,
    num_betas=10,
    num_expression_coeffs=10,
    ext='npz'
).to(device)

print(f"✅ SMPL-X loaded")
print(f"   Vertices: {body_model.get_num_verts()}")
print(f"   Joints: {body_model.NUM_JOINTS}")

---
# STEP 5: Camera Estimation (Iterative PnP)
---

In [ ]:
#@title 5.1 Iterative PnP Camera Estimation
#@markdown This is the key improvement - estimates camera pose accurately

import cv2
import numpy as np
import torch

# COCO to SMPL-X joint mapping (reliable joints only, skip head)
COCO_TO_SMPLX = {
    5: 16,   # left_shoulder
    6: 17,   # right_shoulder
    7: 18,   # left_elbow
    8: 19,   # right_elbow
    9: 20,   # left_wrist
    10: 21,  # right_wrist
    11: 1,   # left_hip
    12: 2,   # right_hip
    13: 4,   # left_knee
    14: 5,   # right_knee
    15: 7,   # left_ankle
    16: 8,   # right_ankle
}

def get_smplx_joints(body_model, betas, device):
    """Get 3D joints from SMPL-X"""
    with torch.no_grad():
        output = body_model(
            betas=betas,
            body_pose=torch.zeros(1, 63, device=device),
            global_orient=torch.zeros(1, 3, device=device)
        )
    return output.joints[0].cpu().numpy()

def solve_pnp(keypoints_2d, joints_3d, K):
    """Solve PnP for camera pose"""
    pts_2d = []
    pts_3d = []
    
    for coco_idx, smplx_idx in COCO_TO_SMPLX.items():
        conf = keypoints_2d[coco_idx, 2]
        if conf > 0.5:
            pts_2d.append(keypoints_2d[coco_idx, :2])
            pts_3d.append(joints_3d[smplx_idx])
    
    if len(pts_2d) < 6:
        return None, None, False
    
    pts_2d = np.array(pts_2d, dtype=np.float64)
    pts_3d = np.array(pts_3d, dtype=np.float64)
    
    success, rvec, tvec, _ = cv2.solvePnPRansac(
        pts_3d, pts_2d, K, None,
        iterationsCount=100,
        reprojectionError=8.0
    )
    
    if not success:
        return None, None, False
    
    R, _ = cv2.Rodrigues(rvec)
    return R, tvec.flatten(), True

# Camera intrinsics
focal = max(VIDEO_INFO['width'], VIDEO_INFO['height'])
K = np.array([
    [focal, 0, VIDEO_INFO['width'] / 2],
    [0, focal, VIDEO_INFO['height'] / 2],
    [0, 0, 1]
], dtype=np.float64)

# Iterative PnP: 3 rounds
print("📷 Iterative PnP Camera Estimation (3 rounds)...")

betas = torch.zeros(1, 10, device=device)
cameras = [None] * len(pose_results)

for iteration in range(3):
    print(f"\n   Round {iteration + 1}/3:")
    
    # Get 3D joints with current shape
    joints_3d = get_smplx_joints(body_model, betas, device)
    
    # Estimate camera for each frame
    success_count = 0
    for i, pose in enumerate(pose_results):
        if pose is None:
            continue
        
        R, t, success = solve_pnp(pose['keypoints'], joints_3d, K)
        
        if success:
            cameras[i] = {'R': R, 't': t, 'K': K.copy()}
            success_count += 1
    
    print(f"      PnP success: {success_count}/{len([p for p in pose_results if p])}")
    
    # Quick shape update (except last iteration)
    if iteration < 2:
        betas_opt = betas.clone().detach().requires_grad_(True)
        optimizer = torch.optim.Adam([betas_opt], lr=0.05)
        
        for _ in range(30):
            optimizer.zero_grad()
            output = body_model(
                betas=betas_opt,
                body_pose=torch.zeros(1, 63, device=device),
                global_orient=torch.zeros(1, 3, device=device)
            )
            joints = output.joints[0]
            
            loss = 0
            for pose, cam in zip(pose_results, cameras):
                if pose is None or cam is None:
                    continue
                # Simple reprojection loss
                R_t = torch.tensor(cam['R'], dtype=torch.float32, device=device)
                t_t = torch.tensor(cam['t'], dtype=torch.float32, device=device)
                K_t = torch.tensor(cam['K'], dtype=torch.float32, device=device)
                
                body_joints = torch.stack([joints[COCO_TO_SMPLX[i]] for i in COCO_TO_SMPLX.keys()])
                cam_pts = torch.matmul(body_joints, R_t.T) + t_t
                proj = torch.matmul(cam_pts, K_t.T)
                proj_2d = proj[:, :2] / (proj[:, 2:3] + 1e-8)
                
                gt_2d = torch.tensor(
                    pose['keypoints'][list(COCO_TO_SMPLX.keys()), :2],
                    dtype=torch.float32, device=device
                )
                loss += torch.mean((proj_2d - gt_2d) ** 2)
            
            loss += 0.01 * torch.mean(betas_opt ** 2)
            loss.backward()
            optimizer.step()
        
        betas = betas_opt.detach()
        print(f"      Shape updated (β0={betas[0,0].item():.3f})")

print(f"\n✅ Camera estimation complete")

---
# STEP 6: Final Shape Optimization
---

In [ ]:
#@title 6.1 Full Shape Optimization with Priors
#@markdown Optimizes body shape using all views with symmetry constraints

N_ITERATIONS = 200  #@param {type:"slider", min:100, max:500, step:50}

# Initialize with refined betas from PnP
betas_final = betas.clone().detach().requires_grad_(True)
optimizer = torch.optim.Adam([betas_final], lr=0.02)

print(f"🔧 Final shape optimization ({N_ITERATIONS} iterations)...")

losses_history = []

for iteration in range(N_ITERATIONS):
    optimizer.zero_grad()
    
    # Forward pass
    output = body_model(
        betas=betas_final,
        body_pose=torch.zeros(1, 63, device=device),
        global_orient=torch.zeros(1, 3, device=device)
    )
    joints = output.joints[0]
    
    # Keypoint reprojection loss
    kp_loss = 0
    count = 0
    
    for pose, cam in zip(pose_results, cameras):
        if pose is None or cam is None:
            continue
        
        R_t = torch.tensor(cam['R'], dtype=torch.float32, device=device)
        t_t = torch.tensor(cam['t'], dtype=torch.float32, device=device)
        K_t = torch.tensor(cam['K'], dtype=torch.float32, device=device)
        
        # Project SMPL-X joints
        body_joints = torch.stack([joints[COCO_TO_SMPLX[i]] for i in COCO_TO_SMPLX.keys()])
        cam_pts = torch.matmul(body_joints, R_t.T) + t_t
        proj = torch.matmul(cam_pts, K_t.T)
        proj_2d = proj[:, :2] / (proj[:, 2:3] + 1e-8)
        
        # Ground truth
        gt_2d = torch.tensor(
            pose['keypoints'][list(COCO_TO_SMPLX.keys()), :2],
            dtype=torch.float32, device=device
        )
        conf = torch.tensor(
            pose['keypoints'][list(COCO_TO_SMPLX.keys()), 2],
            dtype=torch.float32, device=device
        )
        
        diff = proj_2d - gt_2d
        kp_loss += torch.sum(conf.unsqueeze(-1) * diff ** 2)
        count += 1
    
    kp_loss = kp_loss / (count + 1e-8)
    
    # Symmetry loss (left arm ≈ right arm, left leg ≈ right leg)
    left_arm = torch.norm(joints[16] - joints[18]) + torch.norm(joints[18] - joints[20])
    right_arm = torch.norm(joints[17] - joints[19]) + torch.norm(joints[19] - joints[21])
    left_leg = torch.norm(joints[1] - joints[4]) + torch.norm(joints[4] - joints[7])
    right_leg = torch.norm(joints[2] - joints[5]) + torch.norm(joints[5] - joints[8])
    
    symmetry_loss = (left_arm - right_arm) ** 2 + (left_leg - right_leg) ** 2
    
    # Shape regularization
    shape_loss = torch.mean(betas_final ** 2)
    
    # Total loss
    total_loss = kp_loss + 0.1 * symmetry_loss + 0.01 * shape_loss
    
    total_loss.backward()
    optimizer.step()
    
    losses_history.append(total_loss.item())
    
    if iteration % 50 == 0:
        print(f"   Iter {iteration}: Loss = {total_loss.item():.4f}")

print(f"\n✅ Optimization complete (Final loss: {total_loss.item():.4f})")

In [ ]:
#@title 6.2 Plot Optimization Progress
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(losses_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Shape Optimization Progress')
plt.yscale('log')
plt.grid(True)
plt.show()

---
# STEP 7: Extract Body Measurements
---

In [ ]:
#@title 7.1 Generate Canonical Body Mesh

with torch.no_grad():
    output = body_model(
        betas=betas_final,
        body_pose=torch.zeros(1, 63, device=device),
        global_orient=torch.zeros(1, 3, device=device),
        return_verts=True
    )

vertices = output.vertices[0].cpu().numpy()
joints = output.joints[0].cpu().numpy()
faces = body_model.faces

print(f"✅ Canonical mesh generated")
print(f"   Vertices: {vertices.shape}")
print(f"   Joints: {joints.shape}")

In [ ]:
#@title 7.2 Extract Measurements (PCA Circumferences)
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull

def measure_circumference_pca(vertices, center, radius=0.1, scale=100):
    """Measure circumference using PCA plane"""
    distances = np.linalg.norm(vertices - center, axis=1)
    nearby = vertices[distances < radius]
    
    if len(nearby) < 20:
        radius *= 1.5
        nearby = vertices[distances < radius]
    
    if len(nearby) < 10:
        return 0.0
    
    pca = PCA(n_components=2)
    points_2d = pca.fit_transform(nearby - center)
    
    try:
        hull = ConvexHull(points_2d)
        hull_pts = points_2d[hull.vertices]
        
        perimeter = 0
        for i in range(len(hull_pts)):
            perimeter += np.linalg.norm(hull_pts[i] - hull_pts[(i+1) % len(hull_pts)])
        
        return perimeter * scale
    except:
        return 0.0

# Known height for calibration (optional)
KNOWN_HEIGHT_CM = None  #@param {type:"number"}

# Scale factor
raw_height = vertices[:, 1].max() - vertices[:, 1].min()
scale = KNOWN_HEIGHT_CM / raw_height if KNOWN_HEIGHT_CM else 100

# Extract measurements
measurements = {}

# Linear measurements
measurements['height'] = raw_height * scale
measurements['shoulder_width'] = np.linalg.norm(joints[16] - joints[17]) * scale
measurements['hip_width'] = np.linalg.norm(joints[1] - joints[2]) * scale
measurements['torso_length'] = np.linalg.norm(joints[12] - joints[0]) * scale

# Arm length (averaged)
left_arm = np.linalg.norm(joints[16] - joints[18]) + np.linalg.norm(joints[18] - joints[20])
right_arm = np.linalg.norm(joints[17] - joints[19]) + np.linalg.norm(joints[19] - joints[21])
measurements['arm_length'] = ((left_arm + right_arm) / 2) * scale

# Leg length (averaged)
left_leg = np.linalg.norm(joints[1] - joints[4]) + np.linalg.norm(joints[4] - joints[7])
right_leg = np.linalg.norm(joints[2] - joints[5]) + np.linalg.norm(joints[5] - joints[8])
measurements['leg_length'] = ((left_leg + right_leg) / 2) * scale

# Inseam
crotch = (joints[1] + joints[2]) / 2
crotch[1] -= 0.03
ankle = (joints[7] + joints[8]) / 2
measurements['inseam'] = np.linalg.norm(crotch - ankle) * scale

# Circumferences (PCA-based)
chest_center = (joints[16] + joints[17]) / 2
chest_center[1] -= 0.05
measurements['chest_circumference'] = measure_circumference_pca(vertices, chest_center, 0.12, scale)

waist_center = (joints[3] + joints[6]) / 2
measurements['waist_circumference'] = measure_circumference_pca(vertices, waist_center, 0.10, scale)

hip_center = joints[0].copy()
measurements['hip_circumference'] = measure_circumference_pca(vertices, hip_center, 0.12, scale)

# Display results
print("\n" + "="*60)
print("📏 BODY MEASUREMENTS")
print("="*60)
for name, value in measurements.items():
    print(f"   {name.replace('_', ' ').title():<25} {value:>8.1f} cm")
print("="*60)

In [ ]:
#@title 7.3 Visualize Body Model
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(16, 6))

# Front view
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(vertices[::10, 0], vertices[::10, 2], vertices[::10, 1], 
           c='lightblue', s=1, alpha=0.5)
ax1.scatter(joints[:22, 0], joints[:22, 2], joints[:22, 1], c='red', s=50)
ax1.set_title('Front View')
ax1.set_xlabel('X')
ax1.set_ylabel('Z')
ax1.set_zlabel('Y')

# Side view
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(vertices[::10, 2], vertices[::10, 0], vertices[::10, 1],
           c='lightblue', s=1, alpha=0.5)
ax2.scatter(joints[:22, 2], joints[:22, 0], joints[:22, 1], c='red', s=50)
ax2.set_title('Side View')
ax2.view_init(elev=0, azim=0)

# Measurements
ax3 = fig.add_subplot(133)
ax3.axis('off')
text = "📏 MEASUREMENTS\n" + "="*30 + "\n\n"
for name, value in measurements.items():
    text += f"{name.replace('_', ' ').title()}: {value:.1f} cm\n"
ax3.text(0.1, 0.9, text, transform=ax3.transAxes, fontsize=12,
        verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('workspace/output/final/body_measurements.png', dpi=150)
plt.show()

---
# STEP 8: Export Results
---

In [ ]:
#@title 8.1 Save Results
import json
import numpy as np

# Save measurements JSON
output_data = {
    'measurements_cm': {k: float(v) for k, v in measurements.items()},
    'pipeline_version': '2.1-fixed',
    'betas': betas_final.cpu().numpy().tolist()
}

with open('workspace/output/final/measurements.json', 'w') as f:
    json.dump(output_data, f, indent=2)

# Save mesh OBJ
with open('workspace/output/final/body.obj', 'w') as f:
    for v in vertices:
        f.write(f"v {v[0]} {v[1]} {v[2]}\n")
    for face in faces:
        f.write(f"f {face[0]+1} {face[1]+1} {face[2]+1}\n")

# Save numpy data
np.savez('workspace/output/final/body_data.npz',
         vertices=vertices, joints=joints,
         betas=betas_final.cpu().numpy())

print("✅ Results saved!")
print("   - measurements.json")
print("   - body.obj")
print("   - body_data.npz")
print("   - body_measurements.png")

In [ ]:
#@title 8.2 Download Results
from google.colab import files
import shutil

# Create zip
shutil.make_archive('body_reconstruction_results', 'zip', 'workspace/output/final')

# Download
files.download('body_reconstruction_results.zip')

print("\n📥 Download started!")

---
# 🎉 Complete!
---

## Results

You now have:
- **measurements.json** - All body measurements in cm
- **body.obj** - 3D mesh in T-pose (can view in Blender, MeshLab, etc.)
- **body_data.npz** - Raw numpy data for further processing
- **body_measurements.png** - Visualization

## Expected Accuracy

| Measurement | Expected Error |
|-------------|----------------|
| Height | ±1.0-1.5 cm |
| Chest/Waist/Hip | ±2-3 cm |
| Inseam | ±1.5-2 cm |
| Arm/Leg length | ±1.5 cm |

## Tips for Better Results

1. **Known height**: If you know the person's height, set `KNOWN_HEIGHT_CM` for calibration
2. **Better video**: Slow rotation, good lighting, tight clothing
3. **More frames**: Increase `N_FRAMES` to 12-16 for complex poses